In [1]:
import numpy as np
import pandas as pd
from itertools import product

In [2]:
# what varies for all data sets
# n_classes_dep_var = 
# n_ind_vars =
# n_classes_ind_vars = 
# n_samples = 

def generate_data(n_classes_dep_var, n_ind_vars, n_categorical_vars, n_classes_ind_vars, n_samples):
    
    # set coefficients for the model
    coef = [3, 2, 2, 1, 1, 0.5, 0.5]
    coef = coef[:n_ind_vars]

    # generate n x p array for independent variables (predictors)
    X = np.random.uniform(0, 1, (n_samples, n_ind_vars))

    # convert independent continuous variables into categorical ones
    if n_categorical_vars == 0:
        pass
    elif n_categorical_vars >= n_ind_vars:
        return None
    else:
        for i in range(1, n_categorical_vars + 1):
            if n_classes_ind_vars == 2:
                X[:, -i] = np.where(X[:, -i] < np.quantile(X[:, -i], 0.5), 0, 1)  # 0 and 1 for binary
            elif n_classes_ind_vars == 3:
                X[:, -i] = np.where(X[:, -i] < np.quantile(X[:, -i], 0.25), 0,
                                    np.where(X[:, -i] > np.quantile(X[:, -i], 0.75), 2, 1))  # 0, 1, 2 for ternary
            
        

    # generate error term
    e = np.random.normal(0, 1, n_samples)

    # calculate y continuous
    y_continuous = 1 + np.dot(X, coef) + e

    # convert y to categorical
    # keep working from here
    if n_classes_dep_var == 2:
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.5), 'A', 'B')
    elif n_classes_dep_var == 3: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous > np.quantile(y_continuous, 0.75), 'B', 'C'))
    elif n_classes_dep_var == 4: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.5), 'B',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.75), 'C', 'D')))

    # create data that is to be returned
    data_mat = np.concatenate((X, y_continuous.reshape(-1, 1), y_categorical.reshape(-1, 1)), axis = 1)
    data = pd.DataFrame(data_mat)
    # Rename columns to X1, X2, ..., y_cont, y_cat
    data.columns = [f'X{i+1}' for i in range(n_ind_vars)] + ['y_cont', 'y_cat']

    return(data)


In [20]:
# try an example
data = generate_data(3, 5, 3, 3, 100)
print(data)

                      X1                   X2   X3   X4   X5  \
0   0.058641928719343484   0.9196403272887104  2.0  2.0  0.0   
1     0.6524890599671698   0.9156442084771608  2.0  2.0  2.0   
2     0.2636551831533047   0.6793030587858463  1.0  1.0  2.0   
3     0.3297686616023088  0.14619070310196614  2.0  1.0  2.0   
4    0.11636701798224547    0.586482747068296  1.0  1.0  1.0   
..                   ...                  ...  ...  ...  ...   
95   0.38951128544115077   0.7779703736085523  1.0  1.0  0.0   
96  0.010464055804651817   0.2759139379361064  1.0  0.0  1.0   
97   0.38506315507779876   0.7072142507924651  1.0  1.0  1.0   
98   0.25241491204717015   0.5383536147146825  1.0  1.0  2.0   
99    0.5998020603443885  0.13554692949489422  1.0  0.0  0.0   

                y_cont y_cat  
0    7.273206848971284     C  
1   13.103551206945495     B  
2    8.980446985541311     B  
3    7.195663564366999     C  
4    6.644832013984108     C  
..                 ...   ...  
95   7.4558681

In [7]:
n_classes_dep_var = [2, 3, 4]
n_ind_vars = [3, 5, 7]
n_categorical_vars = np.arange(7)
n_classes_ind_vars = [2, 3]
n_samples = [100, 500, 1000, 10000]

In [19]:
# Generate and save datasets
np.random.seed(3103)
for n_classes_dep, n_ind, n_categorical, n_classes_ind, n_samp in product(n_classes_dep_var, n_ind_vars, n_categorical_vars, n_classes_ind_vars, n_samples):
    data = generate_data(n_classes_dep, n_ind, n_categorical, n_classes_ind, n_samp)
    if data is not None:  # Avoid saving if no data is generated
        if n_categorical == 0:   # Avoid saving twice if all X are continuous
            filename = f'../dat/{n_categorical}_0_{n_ind}_{n_classes_dep}_{n_samp}_samples.csv'
            data.to_csv(filename, index=False)
        else:
            filename = f'../dat/{n_categorical}_{n_classes_ind}_{n_ind}_{n_classes_dep}_{n_samp}_samples.csv'
            data.to_csv(filename, index=False)